[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/SD-T2I-360PanoImage-colab/blob/main/SD_T2I_360PanoImage_gradio_colab.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/SD-T2I-360PanoImage
%cd /content/SD-T2I-360PanoImage
!git clone https://huggingface.co/camenduru/sd-t2i-360panoimage /content/SD-T2I-360PanoImage/models
!pip install diffusers transformers accelerate realesrgan==0.3.0
!pip install -q https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl
!pip install /content/SD-T2I-360PanoImage
!pip install -q gradio gradio-pannellum

%cd /content/SD-T2I-360PanoImage
import torch
from txt2panoimg import Text2360PanoramaImagePipeline
txt2panoimg = Text2360PanoramaImagePipeline('./models', torch_dtype=torch.float16, enable_xformers_memory_efficient_attention=True)

import gradio as gr
from gradio_pannellum import Pannellum

def generate(prompt):
  input = {'prompt': prompt, 'upscale': False}
  output = txt2panoimg(input)
  output.save('result.png')
  return output

with gr.Blocks(css="style.css", title=f"Realtime Latent Consistency Model") as demo:
    with gr.Column():
      textbox = gr.Textbox(show_label=False, value="The living room")
      button = gr.Button(value="Generate")

    with gr.Row(variant="default"):
      output_image = Pannellum(show_label=False, interactive=True, show_download_button=True)

    button.click(fn=generate, inputs=[textbox], outputs=[output_image], show_progress=True)

demo.launch(inline=False, share=True, debug=True)